In [22]:
#!pip install gitpython

In [26]:
import os
from pathlib import Path
import json
from git import Repo

In [1]:
madhava = "@madhava@syft-cache.org"

In [15]:
HERE=!pwd
HERE = Path(HERE[0])
HERE

PosixPath('/Users/madhavajay/dev/syft')

In [18]:
SYFT_HOME = HERE / "test_env"
SYFT_HOME

PosixPath('/Users/madhavajay/dev/syft/test_env')

In [141]:
READ_PERMISSION = "r"
WRITE_PERMISSION = "w"

In [176]:
default_permissions = {READ_PERMISSION:[], WRITE_PERMISSION: []}

In [177]:
def git_repo_path_for_user(user, syft_home=SYFT_HOME):
    return syft_home / user

In [178]:
def git_repo_for_user(user, syft_home=SYFT_HOME):
    git_path = git_repo_path_for_user(user, syft_home=syft_home)
    repo = Repo(git_path)
    return repo

In [179]:
madhava_repo = git_repo_for_user(madhava)
repo = madhava_repo

In [180]:
# diff = madhava_repo.git.diff('--cached')

In [207]:
def user_permission_allowed(permissions, user, permission):
    perm = permissions[permission]
    return user in perm or "*" in perm

In [208]:
def validate_perm_file(perm_file_path, user):
    # trigger warning
    with open(perm_file_path, "r") as f:
        perms = default_permissions
        try:
            json_data = json.loads(f.read())
        except Exception as e:
            json_data = perms
        perms.update(json_data)
        
    allowed = user_permission_allowed(perms, user, WRITE_PERMISSION)
    if not allowed:
        file_path = str(perm_file_path).replace(".syftperm", "")
        raise Exception(f"User: {user} does not have {WRITE_PERMISSION} for {file_path}")

In [209]:
def validate_diffs(file_diffs, user) -> bool:
    # split perm and non perm folder / files from the diff
    permission_file_paths = {}
    actual_file_paths = {}
    
    # build the two dicts
    for diff in file_diffs:
        path = diff.b_path
        if path.endswith(".syftperm"):
            actual = path.replace(".syftperm", "")
            if path not in permission_file_paths:
                permission_file_paths[path] = actual
            if path not in actual_file_paths:
                actual_file_paths[actual] = path
        else:
            perm = path + ".syftperm"
            if path not in permission_file_paths:
                permission_file_paths[perm] = path
            if path not in actual_file_paths:
                actual_file_paths[path] = perm
    user_path = git_repo_path_for_user(user)
    for path in actual_file_paths.keys():
        file_path = user_path / Path(path)
        exists = True if os.path.exists(file_path) else False
        if not exists:
            raise Exception(f"File missing: {file_path}")
        is_dir = True if os.path.isdir(file_path) else False
        name = "folder" if is_dir else "file"
        print(f"Validating {name}: {path}")
        perm_file = actual_file_paths[path]
        perm_file_path = user_path / Path(perm_file)
        if not os.path.exists(perm_file_path):
            raise Exception(f"Missing perm file for user: {path} at: {user_path}/{perm_file}")

        validate_perm_file(perm_file_path, user)

In [212]:
diff = repo.git.diff('--cached')

In [214]:
def make_patch(user):
    repo = git_repo_for_user(user)
    file_diffs = repo.head.commit.diff(None)
    validate_diffs(file_diffs, user)
    diff = repo.git.diff('--cached')
    with open(f"./{user}.patch", "w") as f:
        f.write(diff)

In [215]:
make_patch(madhava)

Validating folder: store/prompts
Validating file: store/prompts/prompts.csv
